### Pipeline for QnA with Memory

In [142]:
####Notes####
# chunked data length can be atmost 16 for azureopenai

In [143]:
# !pip3 install langchain
# !pip install openai
# !pip3 install openai chromadb
# !pip3 install tiktoken

In [144]:
# !pip3 install unstructured
# !pip3 install pdf2image
# !pip3 install pdfminer
# !pip3 install pdfminer.six
# !pip3 install pymupdf 
# !pip3 install lxml
# !pip3 install lxml

In [145]:
import os
import sys
import glob
import re
import importlib
import langchain
from langchain.document_loaders import WebBaseLoader, UnstructuredPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
sys.path.append(os.path.join(os.getcwd(), '../scripts'))
import helpers as h
import constants as c
import scraper as sc
importlib.reload(h)
importlib.reload(c)
importlib.reload(sc)

<module 'scraper' from '/Users/rachitjoshi/Documents/Finbot/notebooks/../scripts/scraper.py'>

#### Environment

In [146]:
os.environ["OPENAI_API_KEY"] = "6cdb659e5a9d402e80c212fe8ea26483"

In [147]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://test-chatgpt-flomoney.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

#### Paths

In [148]:
pdfs_path = os.path.join(os.getcwd(), '../data/pdfs')
merged_pdfs_path = os.path.join(os.getcwd(), '../data/pdfs_processed/merged')

#### Data

In [149]:
investorcom_pdfs = [ _ for _ in glob.glob(os.path.join(pdfs_path, '*.pdf')) if 'investorcom' in _]

In [150]:
web_list = sc.scrape_site(
    "https://www.economist.com/",
    c.header_template
)
web_list

['https://www.instagram.com/theeconomist',
 'https://www.economist.com//finance-and-economics/2023/09/25/why-china-may-struggle-to-escape-stagnation',
 'https://www.economist.com//international/2023/10/03/africas-coups-are-part-of-a-far-bigger-crisis',
 'https://www.economist.com//special-report/2023/10/02/video-busting-globalisation-myths',
 'https://www.economistgroup.com/esg/modern-slavery-act-statement',
 'https://www.economist.com/1843',
 'https://www.economist.com//graphic-detail/2023/09/28/places-claiming-to-be-centenarian-hotspots-may-just-have-bad-data',
 'https://www.economist.com//europe/2023/09/28/the-eu-is-finally-rebooting-the-enlargement-machine',
 'https://www.economist.com//britain/2023/10/03/the-conservative-party-conference-is-all-about-the-partying',
 'https://www.economist.com//technology-quarterly/2023/09/26/video-in-search-of-forever',
 'https://www.economist.com//',
 'https://www.economist.com//1843',
 'https://www.economist.com//films/2023/10/02/video-busting-g

In [151]:
len(web_list)

167

In [152]:
len(set(web_list))

167

In [153]:
web_list[35]

'https://www.economist.com//get-the-app'

In [154]:
# web_list = [
#     'https://www.scientificamerican.com/article/pangaea-ultima-the-next-supercontinent-may-doom-mammals-to-far-future-extinction/',
#     'https://www.scientificamerican.com/article/ho-oleilana-a-billion-light-year-wide-bubble-of-galaxies-astounds-astronomers/',
#     'https://www.scientificamerican.com/article/dead-trees-and-a-mysterious-cosmic-explosion-reveal-bigger-quake-risk-for-seattle/',
#     'https://www.scientificamerican.com/article/streetlights-are-mysteriously-turning-purple-heres-why/',
#     'https://www.scientificamerican.com/article/to-move-fast-quantum-maze-solvers-must-forget-the-past/',
# ]

##### - Load

In [155]:
# loaded_data = []
# for pdf in investorcom_pdfs :
#     loaded_data.extend(UnstructuredPDFLoader(file_path=pdf).load())        

In [156]:
loaded_data = []
for page in web_list[:40]:
    try:
        loaded_data.extend(WebBaseLoader(page).load())
    except:
        continue

In [157]:
len(loaded_data)

40

In [158]:
print(loaded_data[17].metadata)

{'source': 'https://www.economist.com//the-world-in-brief', 'title': 'The world in brief | The Economist', 'description': 'Catch up quickly on the global stories that matter.', 'language': 'en'}


In [159]:
loaded_data[17].page_content

'The world in brief | The EconomistThe EconomistThe EconomistSkip to contentMenuWeekly editionThe world in briefSearchLog inOpinionLeadersLetters to the editorBy InvitationCurrent topicsWar in UkraineSummer readsClimate changeCoronavirusThe Biden presidencyThe world economyThe Economist explainsArtificial intelligenceCurrent topicsWar in UkraineSummer readsClimate changeCoronavirusThe Biden presidencyThe world economyThe Economist explainsArtificial intelligenceWorldThe world this weekChinaUnited StatesEuropeBritainMiddle East & AfricaAsiaThe AmericasInternationalA-Z of international relationsA-Z of military termsIn depthScience & technologyGraphic detailSpecial reportsTechnology QuarterlyThe World AheadBriefingEssaySchools briefBusiness & economicsFinance & economicsBusinessBig Mac indexA-Z of economicsEconomic & financial indicatorsCulture & society1843 magazineCultureObituaryThe Economist readsChristmas SpecialsMorePodcastsNewslettersFilmsThe Economist appSubscriber eventsOnline cou

In [160]:
# for pdf in pdf_list :
#     try :
#         pdf_data.extend(UnstructuredPDFLoader(file_path=pdf).load())        
#     except NameError:
#         pdf_loader = UnstructuredPDFLoader(file_path=pdf)
#         pdf_data = pdf_loader.load()

##### - Split

In [161]:
# chunk_size = c.prompt_max//c.retrieval_kwargs['k']
chunk_size = 4000
chunk_overlap = 0

In [162]:
data_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size, chunk_overlap = chunk_overlap
)

In [163]:
chunked_data = data_splitter.split_documents(loaded_data)

In [164]:
len(chunked_data)

87

In [165]:
len(chunked_data[0].page_content)

309

In [166]:
chunked_data[30].page_content

'Customer service is getting worse—and so are customersThe EconomistThe EconomistSkip to contentMenuWeekly editionThe world in briefSearchLog inOpinionLeadersLetters to the editorBy InvitationCurrent topicsWar in UkraineSummer readsClimate changeCoronavirusThe Biden presidencyThe world economyThe Economist explainsArtificial intelligenceCurrent topicsWar in UkraineSummer readsClimate changeCoronavirusThe Biden presidencyThe world economyThe Economist explainsArtificial intelligenceWorldThe world this weekChinaUnited StatesEuropeBritainMiddle East & AfricaAsiaThe AmericasInternationalA-Z of international relationsA-Z of military termsIn depthScience & technologyGraphic detailSpecial reportsTechnology QuarterlyThe World AheadBriefingEssaySchools briefBusiness & economicsFinance & economicsBusinessBig Mac indexA-Z of economicsEconomic & financial indicatorsCulture & society1843 magazineCultureObituaryThe Economist readsChristmas SpecialsMorePodcastsNewslettersFilmsThe Economist appSubscri

##### - Store

In [167]:
chunks_max = 15

In [168]:
vectorstore_engine = 'Finbot-embedding-2'

In [169]:
embedding_model = OpenAIEmbeddings(deployment=vectorstore_engine)

In [170]:
try:
    vectorstore.delete_collection()
except (ValueError, NameError):
    pass
for _ in range(0, len(chunked_data), chunks_max):
    vectorstore = Chroma.from_documents(
        documents=chunked_data[_: _ + chunks_max], embedding=embedding_model
    )

In [171]:
len(vectorstore.get()['documents'])

87

In [172]:
vectorstore.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents'])

##### - Retrieve

In [173]:
question = "tell me about the world in brief?"
docs = vectorstore.similarity_search_with_relevance_scores(question, k=7)
len(docs)

7

In [174]:
len(docs[0][0].page_content)

3996

In [175]:
# print(docs[1][0].page_content)

##### - Generate

In [176]:
llm_engine = 'finbot-gpt'
llm_model = 'text-davinci-002'
temperature = 0
search_type = 'mmr'
retrieval_kwargs = {'k': 5, 'lambda_mult': 0.5, 'fetch_k':10}
answer_max_tokens = 512
save_folder = '../data/pdfs/'
prompt_template_file = os.path.join(os.getcwd(), '../scripts/prompt_template.txt')
pdf_list = [
    _ for _ in glob.glob(os.path.join(os.getcwd(), save_folder, '*.pdf'))
]
web_list = []
langchain.debug=False

In [177]:
############

In [178]:
base_llm = AzureOpenAI(
    engine=llm_engine, 
    model_name=llm_model, 
    temperature=temperature, 
    max_tokens=answer_max_tokens
)

In [179]:
langchain.debug=False

In [180]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [181]:
condense_question_template = os.path.join(
    os.getcwd(), '..', 'scripts/condense_question_template.txt'
)
condense_question_input = c.condense_question_input

In [182]:
chat_prompt = PromptTemplate.from_file(
    condense_question_template,
    input_variables=condense_question_input
)

In [183]:
qna_prompt_template = os.path.join(os.getcwd(), '..', 'scripts/qna_prompt_template.txt')
qna_prompt_input = c.qna_prompt_input
prompt_role = c.prompt_role

In [184]:
qna_prompt = PromptTemplate.from_file(
    qna_prompt_template,
    input_variables=qna_prompt_input,
    partial_variables={'role':prompt_role}
)

In [185]:
history_tokens = 2000
memory = ConversationSummaryBufferMemory(
    llm=base_llm, 
    memory_key="chat_history", 
    return_messages=True, 
    max_token_limit=history_tokens
)

In [186]:
qna_chain = ConversationalRetrievalChain.from_llm(
    base_llm, 
    retriever=vectorstore.as_retriever(
        search_type=search_type, search_kwargs=retrieval_kwargs
    ),
    memory=memory,
    condense_question_prompt=chat_prompt,
    combine_docs_chain_kwargs=dict(prompt=qna_prompt)
)

In [187]:
question = "tell me about the world in brief?"
results = qna_chain({"question": question})

In [188]:
print(results['answer'])

 Kevin McCarthy was ousted as speaker of America’s House of Representatives in a vote forced by Matt Gaetz, a fellow Republican congressman. The motion to unseat Mr McCarthy was supported by eight Republicans and all 208 Democrats present. Mr Gaetz, a firebrand from Florida, filed the “motion to vacate” on Monday after Mr McCarthy pushed through a bipartisan spending bill. The House has taken such a vote only once before, in 1910; that effort, however, was unsuccessful. Mr McCarthy’s exit has thrown the House into turmoil. The Republican majority has no obvious replacement for him. The judge overseeing Donald Trump’s civil trial for alleged business fraud imposed a gagging order to bar the former president, and others involved in the case, from making public remarks about court staff. He warned that violating the order would result in “serious sanctions”. Mr Trump had denigrated a law clerk on social media; the judge then ordered that his post be deleted. It was removed from Mr Trump’s